# JetX Prediction System - Orchestrator

Bu notebook, JetX tahmin projesi için veri yükleme, model eğitimi, kayıt ve simülasyon işlemlerini yönetir.

**Adımlar:**
1. Ortam Hazırlığı
2. Veri Yükleme
3. Model A Eğitimi (CatBoost)
4. Model B Oluşturma (k-NN Hafıza)
5. Modelleri Kaydetme
6. Simülasyon (Sanal Kasa Testi)

In [ ]:
# 1. Ortam Hazırlığı
!pip install catboost pandas numpy scikit-learn joblib matplotlib

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Proje dizinini path'e ekle
sys.path.append(os.getcwd())

from jetx_project.data_loader import load_data, split_train_test, get_values_array
from jetx_project.model_a import prepare_model_a_data, train_model_a, save_models
from jetx_project.model_b import build_memory, train_model_b, save_memory, predict_model_b, create_pattern_vector, evaluate_model_b
from jetx_project.simulation import run_simulation

In [ ]:
# 2. Veri Yükleme
print("Veritabanından veri yükleniyor...")
df = load_data('jetx.db')
values = get_values_array(df)

print(f"Toplam veri sayısı: {len(values)}")

# Train/Test Ayrımı
train_df, test_df = split_train_test(df)
train_values = get_values_array(train_df)
test_values = get_values_array(test_df)

print(f"Eğitim seti: {len(train_values)}")
print(f"Test seti: {len(test_values)}")

In [ ]:
# 3. Model A Eğitimi (Feature Tabanlı)
print("Model A için veri hazırlanıyor (Feature Extraction)...")
X_train, y_p15_train, y_p3_train, y_x_train = prepare_model_a_data(train_values, start_index=500)

print(f"Feature seti boyutu: {X_train.shape}")

print("Model A eğitiliyor...")
# train_model_a artık validasyon ve early stopping yapıyor
ma_p15, ma_p3, ma_x = train_model_a(X_train, y_p15_train, y_p3_train, y_x_train)

print("Model A eğitimi tamamlandı.")

In [ ]:
# 4. Model B Oluşturma (Pattern Hafızası)
print("Model B hafızası oluşturuluyor...")
patterns, targets = build_memory(train_values, start_index=200)

print(f"Hafızadaki pattern sayısı: {len(patterns)}")

print("k-NN modeli eğitiliyor (indeksleniyor)...")
nbrs = train_model_b(patterns)

print("Model B hazır. Test seti üzerinde performans ölçülüyor...")
evaluate_model_b(nbrs, targets, test_values, start_index=200)

In [ ]:
# 5. Modelleri Kaydetme
print("Modeller kaydediliyor...")
save_models(ma_p15, ma_p3, ma_x, output_dir='.')
save_memory(nbrs, patterns, targets, output_dir='.')

print("Modeller kaydedildi.")

In [ ]:
# 6. Simülasyon (Test Seti Üzerinde)
print("Simülasyon başlıyor...")

test_start_idx = 500
if len(test_values) < test_start_idx + 10:
    print("Test seti simülasyon için çok kısa!")
else:
    # --- Model A Tahminleri ---
    print("Model A tahminleri üretiliyor...")
    X_test, _, _, _ = prepare_model_a_data(test_values, start_index=test_start_idx)
    
    true_vals_A = test_values[test_start_idx+1 : len(test_values)]
    
    pred_p15_A = ma_p15.predict_proba(X_test)[:, 1]
    pred_p3_A = ma_p3.predict_proba(X_test)[:, 1]
    pred_x_A = ma_x.predict(X_test)
    
    df_res_A = pd.DataFrame({
        'true_val': true_vals_A,
        'p_1_5': pred_p15_A,
        'p_3': pred_p3_A,
        'pred_x': pred_x_A
    })
    
    results_A = run_simulation(df_res_A, model_name="Model A")
    
    # --- Model B Tahminleri ---
    print("Model B tahminleri üretiliyor...")
    res_B = []
    for i in range(test_start_idx, len(test_values) - 1):
        pat = create_pattern_vector(test_values, i)
        if pat is not None:
            p15, p3, px = predict_model_b(nbrs, targets, pat)
            res_B.append({
                'true_val': test_values[i+1],
                'p_1_5': p15,
                'p_3': p3,
                'pred_x': px
            })
            
    df_res_B = pd.DataFrame(res_B)
    results_B = run_simulation(df_res_B, model_name="Model B")

    # --- Grafik Çizimi ---
    def plot_bankrolls(results, title):
        plt.figure(figsize=(12, 6))
        plt.plot(results['Kasa1'].history, label='Kasa 1 (1.5x @ 80%)')
        plt.plot(results['Kasa2'].history, label='Kasa 2 (1.5x @ 90%)')
        plt.plot(results['Kasa3'].history, label='Kasa 3 (Dynamic)')
        plt.title(title)
        plt.xlabel('Oyun Sayısı')
        plt.ylabel('Bakiye')
        plt.legend()
        plt.grid(True)
        plt.show()

    plot_bankrolls(results_A, "Model A Kasa Performansı")
    plot_bankrolls(results_B, "Model B Kasa Performansı")